In [6]:
using Clapeyron
model = PCSAFT(["ethanol","water"];userlocations=(;
Mw = [46.069,18.015], 
segment = [2.3827,2.1945],
sigma = [3.1771,2.229],
epsilon = [198.24,141.66],
n_H=[1,2],
n_e=[1,2],
epsilon_assoc = Dict(
    (("ethanol","e"),("ethanol","H")) =>2653.4,
    (("ethanol","H"),("water","e")) =>2228.785,
	(("ethanol","e"),("water","H")) =>2228.785,
    (("water","H"),("water","e")) =>1804.17
    ),
bondvol = Dict(
    (("ethanol","e"),("ethanol","H")) =>0.032384,
    (("ethanol","H"),("water","e")) =>0.07753951,
	(("ethanol","e"),("water","H")) =>0.07753951,
    (("water","H"),("water","e")) =>0.2039
)
))


PCSAFT{BasicIdeal, Float64} with 2 components:
 "ethanol"
 "water"
Contains parameters: Mw, segment, sigma, epsilon, epsilon_assoc, bondvol

In [18]:
function SpeciesFugacityCheck(model::EoSModel, T, p, x)
    numphases, numspecies = size(x)
    Fugacities = zeros(numphases, numspecies)
    for i = 1:numphases
        Fugacities[i,:] = fugacity_coefficient(model, p, T, x[i,:]) .* x[i,:] .* p
    end  
    RelErrorSpecies = zeros(numspecies)
    for j = 1:numspecies
        RelErrorSpecies[j] = (sum(abs(Fugacities[1,j] - Fugacities[i,j])/Fugacities[1,j] for i in 2:numphases))/(numphases-1)
    end  
    normRelError = sum(abs.(RelErrorSpecies)) / numspecies  
    return (normRelError, RelErrorSpecies, Fugacities)
end

result=[]
function insert_points(array)
    result = Float64[]
    for i in 1:length(array)-1
        push!(result, array[i])
        for j in 1:99
            point = array[i] + (array[i+1] - array[i]) / 100 * j
            push!(result, point)
        end
    end
    push!(result, array[end])
    return result
end


insert_points (generic function with 1 method)

In [19]:
T=303.15
P=[4413 4803 5203 5781 6386 7329 8189 8723 9085 9303 9403 9663 9869 9999 10199 10341 10394 10427 10435 10445 10457 10467 10473]

1×23 Matrix{Int64}:
 4413  4803  5203  5781  6386  7329  …  10435  10445  10457  10467  10473

In [20]:
result = insert_points(P)
ϕ = 0.5
x_VLE = zeros(length(result),6)
z0=0.02
P0=P[1]
(x,n,G) = tp_flash(model,P0,T,[z0,1-z0],MichelsenTPFlash(equilibrium=:vle,K0=[9.581,0.96]))
println(x)
(normRelError_1PC, RelErrorSpecies_1PC, Fugacities_1PC) = SpeciesFugacityCheck(model, T, P0, x)

[0.004386569243500269 0.9956134307564998; 0.03399666562770575 0.9660033343722942]


(1.4925390185716655e-10, [2.9717131793791923e-10, 1.3364857764138495e-12], [149.0051505027789 4247.075826721852; 149.00515045849883 4247.075826727528])

In [21]:
K0=[x[2,1]/x[1,1],x[2,2]/x[1,2]]
z0 = x[1,1]*ϕ+x[2,1]*ϕ 
for i=1:length(result)
	(x,n,G) = tp_flash(model,result[i],T,[z0,1-z0],MichelsenTPFlash(equilibrium=:vle,K0=K0))
	(normRelError_1PC, RelErrorSpecies_1PC, Fugacities_1PC) = SpeciesFugacityCheck(model,T, result[i], x)
	x_VLE[i,1:2] = x[1,:]
	x_VLE[i,3:4] = x[2,:]
	x_VLE[i,5]=normRelError_1PC
	x_VLE[i,6] = result[i]
	K0=[x[2,1]/x[1,1],x[2,2]/x[1,2]]
	z0 = x[1,1]*ϕ+x[2,1]*ϕ     
	if x[2,1]-x[1,1] < 1e-3
		break
	end
end


In [27]:
cal=zeros(length(P),4)

for i=1:length(P)
	cal[i,1]=x_VLE[:,1][1+100*(i-1)]
	cal[i,2]=x_VLE[:,3][1+100*(i-1)]
	cal[i,3]=x_VLE[:,5][1+100*(i-1)]
	cal[i,4]=x_VLE[:,6][1+100*(i-1)]
end
cal
x = reduce(vcat, cal[:,1])
y = reduce(vcat, cal[:,2])
e = reduce(vcat, cal[:,3])
p = reduce(vcat, cal[:,4])
println(x)
println(y)


[0.00438656924350027, 0.017860870159970076, 0.03259543067719481, 0.05600257491057398, 0.0841463344457777, 0.13978698711931956, 0.21360768853198528, 0.28116408225340955, 0.3432216936531921, 0.38918756713533026, 0.4127282541787946, 0.4815026413329741, 0.5438329128845488, 0.5868885926321231, 0.6599602923581108, 0.7193744018696225, 0.7442506108728905, 0.7608794333469879, 0.7650761087898775, 0.7704265543905073, 0.7770153818620666, 0.7826617776121455, 0.7861244579518935]
[0.03399666562770576, 0.12389787965969057, 0.2024863716508532, 0.29758374682049865, 0.3797786549889071, 0.4836704202802138, 0.5621249803435602, 0.606959146553133, 0.6373979338288956, 0.6564769006523654, 0.665593030111013, 0.6910401928738792, 0.7140184215163613, 0.7305638643108883, 0.7610657093829805, 0.7891357318523488, 0.8020161985786487, 0.811057072929492, 0.8133969857018526, 0.8164155257118978, 0.8201883815616018, 0.8234717019402507, 0.8255086081456441]